# INSTALAR LIBRERIAS

In [1]:
# !pip install docx2pdf
#pip install ipywidgets

In [2]:
gegeg

NameError: name 'gegeg' is not defined

# IMPORTAR

In [1]:
from docx2pdf import convert
import os
import time

# CONSTRUIR LA RUTA DE LOS ARCHIVOS

In [2]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../../../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [3]:
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
# colocar aqui la ruta de los archivos
path = ruta_completa + r'/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 0 INFORMES
0 INFORMES


[]

# INICIA EL PROCESO DE CONVERCIÓN

In [ ]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

In [ ]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

In [ ]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdf'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdf').split('/')[-1]
    convert(filename, r'C:\Users\bismarksr\Downloads' + pdf)
    i+=1
    time.sleep(5)
print('END')

# Leer archivos

In [5]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [6]:
contenido = os.listdir(ruta_completa + r'/pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
print(len(dic_files), 'informes')
dic_files

17 informes


{10806: '10806_CRS_11-06-2025_PALICIO BRUNO OSCAR ROBERTO_500.pdf',
 12105: '12105_CRS_19-06-2025_ROCA REYES MIGUEL ANGEL_542.pdf',
 13187: '13187_CRS_13-06-2025_SUAREZ CUELLAR REMBERTO_560.pdf',
 1478: '1478_CRS_12-06-2025_CALLE RALDEZ MARCO ANTONIO_527.pdf',
 15085: '15085_CRS_12-06-2025_CRUZ RUTH JACQUELINE B. DE_574.pdf',
 2351: '2351_CRS_19-06-2025_CANDIA RUIZ ADALBERTO_556.pdf',
 2353: '2353_CRS_19-06-2025_CARO BENITES GERMAN_456.pdf',
 2505: '2505_CRS_12-06-2025_CRUZ BARRIENTOS CHRISTIAN FERNANDO_573.pdf',
 2510: '2510_CRS_12-06-2025_CRUZ BARRIENTOS OSCAR ANDREZ_575.pdf',
 40250: '40250_CRS_19-06-2025_AMURRIO MELGAROJO BISMARK ELISEO_482.pdf',
 41591: '41591_CRS_04-06-2025_BENAVIDES GARCIA GABINO_484.pdf',
 42239: '42239_CRS_13-06-2025_CORDOVA OLGUIN LEONARDO_563.pdf',
 42347: '42347_CRS_19-06-2025_MERCADO KATERINE JUSTINIANO DE_518.pdf',
 42378: '42378_CRS_19-06-2025_CANDIA RUIZ ORLANDO_557.pdf',
 6638: '6638_CRS_24-06-2025_CABRERA LOMBARDI LEANDRO JAVIER_580.pdf',
 8678: '8678

# CARGAR LOS PDFS A GOOGLE DRIVE

In [7]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "2DA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta_completa + r'/pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 10806_CRS_11-06-2025_PALICIO BRUNO OSCAR ROBERTO_500.pdf with mimeType application/pdf
Created file 12105_CRS_19-06-2025_ROCA REYES MIGUEL ANGEL_542.pdf with mimeType application/pdf
Created file 13187_CRS_13-06-2025_SUAREZ CUELLAR REMBERTO_560.pdf with mimeType application/pdf
Created file 1478_CRS_12-06-2025_CALLE RALDEZ MARCO ANTONIO_527.pdf with mimeType application/pdf
Created file 15085_CRS_12-06-2025_CRUZ RUTH JACQUELINE B. DE_574.pdf with mimeType application/pdf
Created file 2351_CRS_19-06-2025_CANDIA RUIZ ADALBERTO_556.pdf with mimeType application/pdf
Created file 2353_CRS_19-06-2025_CARO BENITES GERMAN_456.pdf with mimeType application/pdf
Created file 2505_CRS_12-06-2025_CRUZ BARRIENTOS CHRISTIAN FERNANDO_573.pdf with mimeType application/pdf
Created file 2510_CRS_12-06-2025_CRUZ BARRIENTOS OSCAR ANDREZ_575.pdf with mimeType application/pdf
Created file 40250_CRS_19-06-2025_AMURRIO MELGAROJO BISMARK ELISEO_482.pdf with mimeType application/pdf
Created file 415

In [8]:
print(len(dic_urls), 'URLs')
dic_urls

17 URLs


{10806: 'https://drive.google.com/file/d/178IppruoCUyNZVs8aOkiJP1CPXlYWFpk/view?usp=drivesdk',
 12105: 'https://drive.google.com/file/d/1syIgQi1GfzLke_Pl_PvIfJnA9yrJI4SP/view?usp=drivesdk',
 13187: 'https://drive.google.com/file/d/1K_nRGPaLFNc0VQzQrQc4LZdiBgdvQsW3/view?usp=drivesdk',
 1478: 'https://drive.google.com/file/d/1Io5eGo3Flvbo8vfHTqHacZT52EVyCsWV/view?usp=drivesdk',
 15085: 'https://drive.google.com/file/d/1P8QlSw5iulSTfIamCHUVWnst380yDcT6/view?usp=drivesdk',
 2351: 'https://drive.google.com/file/d/1pPz7WU8VkScCGgc3jK_vSYdHLQJ77YPZ/view?usp=drivesdk',
 2353: 'https://drive.google.com/file/d/1pdc-KdBEGAi8oGNZBO3hKYfSp_035Ad5/view?usp=drivesdk',
 2505: 'https://drive.google.com/file/d/12IJV1nKNJWqjdnKTN6NDyIVa9LfrQKHI/view?usp=drivesdk',
 2510: 'https://drive.google.com/file/d/1XhFhJbXuLLnDvqPUCV-u3g16BEU6GYKO/view?usp=drivesdk',
 40250: 'https://drive.google.com/file/d/1DpwYAEKibOnhZ4TE7tDcTz-_VYane--J/view?usp=drivesdk',
 41591: 'https://drive.google.com/file/d/1QSamBOPGtYyYf

# conexion a amigocloud

In [9]:
import sys
sys.path.append('../../../_amigocloud')

from amigocloud import AmigoCloud

In [10]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [11]:
for key in dic_urls:
    query = {'query': 'update dataset_307560 set link_doc_2da = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    print(key)
print('end')

10806
12105
13187
1478
15085
2351
2353
2505
2510
40250
41591
42239
42347
42378
6638
8678
8838
end
